### Pytorch SkipGram

In [1]:
!pip install torchdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━

In [3]:
!pip install portalocker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import torch.nn as nn
from functools import partial
from torch.utils.data import DataLoader
from torchtext.data import to_map_style_dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import WikiText2, WikiText103
import numpy as np

import torch
import torch.nn as nn
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import math


import pandas as pd
from torchtext.vocab import vocab

In [5]:
WINDOW_SIZE = 5
BATCH_SIZE = 256    #"<YOUR CODE>"
EMB_DIM = 256         #"<YOUR CODE>"
EPOCHS = 15          #"<YOUR CODE>"

In [6]:
if torch.cuda.is_available():
    #"<YOUR CODE>"
  device = torch.device('cuda') # Если мы используем GPU
else:
  #"<YOUR CODE>"
  device = torch.device('cpu') # Если мы используем CPU

In [7]:
!wget https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz -O wikitext-2.tar.gz
!tar -xvzf wikitext-2.tar.gz

--2023-05-11 16:14:23--  https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.233.232, 52.216.248.86, 52.217.203.64, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.233.232|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4070055 (3.9M) [application/x-tar]
Saving to: ‘wikitext-2.tar.gz’

wikitext-2.tar.gz   100%[===================>]   3.88M  --.-KB/s    in 0.1s    

2023-05-11 16:14:24 (27.1 MB/s) - ‘wikitext-2.tar.gz’ saved [4070055/4070055]

wikitext-2/
wikitext-2/train.csv
wikitext-2/test.csv


In [8]:
def load_data(filepath):
  with open(filepath) as f:
    return f.readlines()

In [9]:
train = load_data("wikitext-2/train.csv")
test = load_data("wikitext-2/test.csv")
data = train + test

In [10]:
tokenizer = get_tokenizer("basic_english", language="en")

In [11]:
def yield_tokens(data_obj):
  for text in data_obj:
    yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(data), specials=["<unk>"], min_freq=20)
vocab.set_default_index(vocab["<unk>"])

In [12]:
len(vocab)

8627

In [13]:
vocab['asdasdasd']

0

In [14]:
text_pipeline = lambda x: vocab(tokenizer(x))

In [15]:
def build_contexts(row, window_size=3):
  contexts = []
  for i in range(len(row)):
    central_word = row[i]
    context = [row[i + delta] for delta in range(-window_size, window_size + 1) if delta != 0 and i + delta >= 0 and i + delta < len(row)]

    for c_w in context:
      contexts.append((central_word, c_w)) 
    return contexts

In [16]:
class Word2VecDataset(Dataset):
  def __init__(self, data, vocab, wsize=3):
    self.vocab_size = len(vocab)
    self.data = [text_pipeline(item) for item in data]
    self.data = [item for text in self.data for item in text]
    self.data = build_contexts(self.data, window_size=wsize)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [17]:
dataset = Word2VecDataset(data, vocab)
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle= True)
#"<YOUR CODE>"

In [18]:
central_word, context = dataset[0]
central_word, context 

(9, 435)

In [19]:
class SkipGram_Model(nn.Module):
  def __init__(self, vocab_size: int):
    super(SkipGram_Model, self).__init__()
    self.embeddings = nn.Embedding(
        num_embeddings= vocab_size,
        embedding_dim= EMB_DIM
       #"<YOUR CODE>"
    )
    self.linear = nn.Linear(
        in_features= EMB_DIM,
        out_features= vocab_size
        #"<YOUR CODE>"
    )

  def forward(self, inputs_):
    x = self.embeddings(inputs_)
    x = self.linear(x)
    #"<YOUR CODE>"
    return x

In [20]:
vocab_size = len(vocab)
#"<YOUR CODE>"
model = SkipGram_Model(vocab_size)
if torch.cuda.is_available():
  model = model.cuda()

In [21]:
criterion = torch.nn.CrossEntropyLoss()   #"<YOUR CODE>"
optimizer = torch.optim.Adam(model.parameters())   #"<YOUR CODE>"
#optimizer = torch.optim.SGD(model.parameters()) 




In [22]:
from tqdm import tqdm

def train_model(dataloader, model, optimizer, criterion):
  epoch_loss=[]
        
  for word, context in tqdm(train_dataloader):
    optimizer.zero_grad() 
    pred = model(word.to(device))
    loss = criterion (pred, context.to(device))  
    loss.backward()      
    epoch_loss.append(loss.detach().cpu().numpy())
    optimizer.step()
       
    #"<YOUR CODE>"
  return np.mean(epoch_loss) 

In [23]:
for epoch in range(EPOCHS):
  loss = train_model(train_dataloader, model, optimizer, criterion)
  if epoch % 2 == 0:
    print(f'Epoch {epoch}: train loss {loss}')

100%|██████████| 1/1 [00:00<00:00,  4.88it/s]


Epoch 0: train loss 9.017966270446777


100%|██████████| 1/1 [00:00<00:00,  7.91it/s]


Epoch 2: train loss 8.182194709777832


100%|██████████| 1/1 [00:00<00:00,  8.71it/s]


...

Epoch 10: train loss 4.856422424316406


100%|██████████| 1/1 [00:00<00:00, 18.67it/s]


Epoch 12: train loss 4.050792694091797


100%|██████████| 1/1 [00:00<00:00, 15.82it/s]

Epoch 14: train loss 3.2874410152435303


In [24]:
embeddings = list(model.parameters())[0]
embeddings = embeddings.cpu().detach().numpy()

# normalization
norms = (embeddings ** 2).sum(axis=1) ** (1 / 2)
norms = np.reshape(norms, (len(norms), 1))
embeddings_norm = embeddings / norms
embeddings_norm.shape

(8627, 256)

In [25]:
def get_top_similar(word, n=10):
    word_id = vocab[word]
    if word_id == 0:
        print("Out of vocabulary word")
        return

    word_vec = embeddings_norm[word_id]
    word_vec = np.reshape(word_vec, (len(word_vec), 1))
    dists = np.matmul(embeddings_norm, word_vec).flatten()
    top_ids = np.argsort(-dists)[1 : n + 1]

    top_dict = {}
    for sim_word_id in top_ids:
        sim_word = vocab.lookup_token(sim_word_id)
        top_dict[sim_word] = dists[sim_word_id]
    return top_dict

In [26]:
get_top_similar('hero')

{'makes': 0.24810466,
 'aged': 0.22608688,
 'popular': 0.21625467,
 'enclosed': 0.19651905,
 'astronomical': 0.19627309,
 'grain': 0.19179407,
 'efficient': 0.1910615,
 '275': 0.19070694,
 'indu': 0.19019961,
 'layers': 0.18766634}